In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded
import Transmon_specifications as transmon

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import os



# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
# import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
# from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
# from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True
c1, renderer_q3d, q3d = dp.init_q3d_sim()
eig_all, hfss_renderer, hfss = dp.init_hfss_sim()

## Calculate Qubit Params

In [3]:
import Transmon_property as tp

In [4]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from scipy.optimize import root_scalar

In [5]:
ratio = 65
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]
    return transition_frequencies[0]
    
class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = self.qubit_freq_to_cap()
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.freq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        def target_function(L, desired_frequency,):
            self.L = L * u.nH  # Convert L to nanohenrys
            self.calculate_qubit_freq()
            calculated_frequency = self.freq.to(u.GHz).value
            return calculated_frequency - desired_frequency
        result = root_scalar(target_function, args=(desired_frequency.to(u.GHz).value,), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH
    def calculate_qubit_freq(self,):
        L = self.L
        ratio = self.ratio
        EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
        #, equivalencies=u.spectral())  # Convert to GHz
        EC = EJ/ratio
        ng = 0.0   # Offset charge
        nlev = 10  # Number of energy levels to consider

        # Create a Transmon object
        transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

        # Calculate the eigenenergies
        eigenenergies = transmon.eigenvals()

        # Calculate the transition frequencies (in GHz)
        transition_frequencies = np.diff(eigenenergies)

        # Calculate the anharmonicity (in GHz)
        anharmonicity = transition_frequencies[1] - transition_frequencies[0]
        self.freq = transition_frequencies[0]*u.GHz
        self.alpha = anharmonicity*u.GHz
    

Design specifics

In [6]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [7]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [8]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [9]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

## Resonator calculations

In [10]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance
import qiskit_metal.analyses as analyses

In [11]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [12]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        self.C = np.pi/2/self.freq/2/np.pi/self.Z0
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.Cl,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.Cl*=u.F/u.m
        
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

### dispersive readout function

In [13]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def update_g(self):
        self.g = self.calc_g()
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        argv = [res.freq.si.value, 
                c_tr.si.value, 
                res.len.si.value, 
                res.pin.si.value, 
                res.gap.si.value, 
                2]
        
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value, 
                c_tr.si.value, 
                res.len.si.value, 
                res.pin.si.value, 
                res.gap.si.value, 
                2)*u.Hz).to(u.MHz)
    def update_kappa(self):
        self.kappa = self.calc_kappa()
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*self.g**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def update_chi(self):
        self.chi = self.calc_chi()
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = self.g**2*self.kappa/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,update_g = True, update_chi = True):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        if update_g:
            self.update_g()
        if update_chi:
            self.update_chi()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        kappa_val = self.chi*2
        
        purcellGamma = self.g**2*kappa_val/delta**2
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        # if target_kappa is None:
        #     target_kappa = self.chi*2
        # c_tr = 1*u.fF
        # self.c_tr = c_tr
        # self.update_kappa()
        # while self.kappa < target_kappa:
        #     c_tr += .1*u.fF
        #     self.c_tr = c_tr
        #     self.update_kappa()
        # return c_tr
        self.c_tr = 1*u.fF
        return self.c_tr


In [14]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [15]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

## Leonding method

In [16]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params_dim4(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,0,0],
                  [0,1,1,0],
                  [0,1,-1,0],
                  [0,0,0,1]])
    cap_matrix[1,2] -= cj
    cap_matrix[2,1] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[1,1] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[2,2]
    coupling_capacitance = np.reciprocal(inverse)[2,3]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def get_cap_params_dim2(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,1,0],
                  [1,-1,0],
                  [0,0,1]])
    cap_matrix[0,1] -= cj
    cap_matrix[1,0] -= cj
    cap_matrix[1,1] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[1,1]
    coupling_capacitance = np.reciprocal(inverse)[1,2]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def qubit_zpf(E_J, E_c):
    z_Q = (1/2*np.pi)*np.sqrt(8*E_c/E_J)
    return np.sqrt(1/(4*np.pi*z_Q))
def res_zpf(Z_0 = 50*u.Ohm):
    R_Q = c.h/(2*c.e.si**2)
    Z = 2*Z_0/np.pi
    z= Z/R_Q
    return np.sqrt(1/(4*np.pi*z))
    

In [17]:
phi0 = c.h/2/c.e.si
epsilon_r = 11.45
z0 = 50*u.Ohm

def g_leonDing_old(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [18]:
def g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    
    
    # df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    # df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    cap[1,1] += cap[0,1]
    cap[2,2] += cap[0,2]
    cap[3,3] += cap[0,3]
    
    cap[0,3] += res.C.to(u.fF).value
    cap[3,0] += res.C.to(u.fF).value
    cap[0,0] -= res.C.to(u.fF).value
    cap[3,3] -= res.C.to(u.fF).value
    
    
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [19]:
# g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH)

In [20]:
qubit.alpha = 0.2*u.GHz
# dr.calc_g().to(u.MHz)

In [21]:
c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
c1.setup.freq_readout = 6.0
c1.setup.freq_bus = []

c1.run_lom()
c1.lumped_oscillator_all

03:37PM 28s WARNING [run_lom]: Please initialize the capacitance_matrix before executing this method.`self.sim.capacitance_matrix = pd.DataFrame(...)`


{}

## Alex Page Thesis method

In [22]:
df = c1.sim.capacitance_matrix

In [23]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
z0 = 50*u.Ohm


Cq = 20*u.fF
Lj = 13*u.nH
Ec = (c.e.si**2/2/Cq)/c.h


In [24]:
def g_alexPage(c1 = c1,wr = 6*u.GHz, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    c_1 = -df['pad_left_Q1']['resonator_pad_Q1']*u.fF
    c_2 = -df['pad_right_Q1']['ground_main_plane']*u.fF
    c_3 = -df['pad_right_Q1']['resonator_pad_Q1']*u.fF
    c_4 = -df['pad_left_Q1']['ground_main_plane']*u.fF
    ct = -df['pad_left_Q1']['pad_right_Q1']*u.fF

    c_phi = (c_2+c_3)*(c_1+c_4)/(c_1+c_2+c_3+c_4)+2*u.fF+ct
    c_ceff = (c_1**2-c_3*c_4)/(c_1+c_2+c_3+c_4+ct)
    beta = (c_1*c_2-c_3*c_4)/((c_2+c_3)*(c_1+c_4)+ct*(c_1+c_2+c_3+c_4))
    Ec = (c.e.si**2/2/c_phi)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h
    w01 = np.sqrt(8*Ec*Ej)
    delta = wr-w01
    c_res = np.pi/2/wr/z0
    Vrms = np.sqrt(c.hbar*wr/2/c_res)
    g = c.e.si*Vrms*beta/c.hbar*np.sqrt(2)*(Ej/8/Ec)**0.25
    return g.to(u.MHz), Ec.to(u.MHz), Ej.to(u.GHz), w01.to(u.GHz)

In [25]:
# g_alexPage()

## Qiskit Metal

In [26]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [27]:
from datetime import datetime

def get_current_time_string():
    now = datetime.now()
    return now.strftime("%Y-%m-%d%H-%M")

# Example usage
print(get_current_time_string())

2024-12-1015-37


In [28]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)

Optimal inductance L: 13.676453476080171 nH


In [29]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

# Be aware of the default_options that can be overridden by user.
options = TransmonCross.get_template_options(design)

In [33]:
options

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 '_default_connection_pads': {'connector_type': '0',
  'claw_length': '30um',
  'ground_spacing': '5um',
  'claw_width': '10um',
  'claw_gap': '6um',
  'connector_location': '0'},
 'cross_width': '20um',
 'cross_length': '200um',
 'cross_gap': '20um',
 'hfss_inductance': '13nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '13nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

In [49]:
options['connection_pads'] = dict(a = dict(ground_spacing = '11um',claw_gap = '7um',
                                           claw_length = '150um',connector_location = '90', connector_type = '0'),)
options['cross_width'] = '31um'
options['cross_length'] = '195um'
options['orientation'] = '-90'
options['cross_gap'] = '19um'
options['pos_x'] = f'{1.425-3500}um'
options['pos_y'] = f'{5139-3500}um'


# Create a new Transmon Cross object with name 'Q1'
q1 = TransmonCross(design, 'Q1', options=options)

gui.rebuild()  # rebuild the design and plot
gui.autoscale() #resize GUI to see QComponent
gui.zoom_on_components(['Q1']) #Can also gui.zoom_on_components([q1.name])

In [ ]:
options['connection_pads'] = dict(a = dict(ground_spacing = '11um',
                                           claw_gap = '7um',
                                           claw_length = '150um',
                                           connector_location = '90', 
                                           connector_type = '0'),)
options['cross_width'] = '31um'
options['cross_length'] = '195um'
options['orientation'] = '-90'
options['cross_gap'] = '19um'


# Create a new Transmon Cross object with name 'Q1'
q2 = TransmonCross(design, 'Q2', options=options)

gui.rebuild()  # rebuild the design and plot
gui.autoscale() #resize GUI to see QComponent
gui.zoom_on_components(['Q2']) #Can also gui.zoom_on_components([q1.name])